In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

API_KEY = 'f8bfa5faeb48897daf693a147c376473'

if not API_KEY:
    raise ValueError("SCRAPER_API_KEY environment variable not set.")

target_url = "https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B%5D=ti&showCol%5B%5D=fm&showCol%5B%5D=oa&showCol%5B%5D=at&showCol%5B%5D=md&showCol%5B%5D=df&showCol%5B%5D=cw&showCol%5B%5D=ps"
scraper_base = f"http://api.scraperapi.com/?api_key={API_KEY}&render=true&url="
scraper_url = scraper_base + target_url

response = requests.get(scraper_base + target_url)
soup = BeautifulSoup(response.text, 'html.parser')

base_url = "https://sofifa.com"

select_season = soup.find('select', {'name': 'version'})
select_roster = soup.find('select', {'name': 'roster'})


for season_option in select_season.find_all('option'):
    for roster_option in select_roster.find_all('option'):
        relative_url = roster_option.get('value')
        full_url = base_url + relative_url
        date = roster_option.get_text(strip=True)
        safe_date = date.replace("/", "-").replace(":", "-").strip()
        file_path = f'../data/scraping/teams_{safe_date}.csv'
        
        try:
            response = requests.get(scraper_base + full_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            table = soup.find('table')
            if table is None:
                print(f'No table found for: {full_url}')
                continue
  

            rows = table.find_all('tr')
            if not rows or not rows[0].find_all('th'):
                print(f'no headers found: {full_url}')
                continue

            headers = [th.get_text(strip=True) for th in rows[0].find_all('th')]
            data = []
        
        
            for row in rows[1:]:
                cols = [td.get_text(strip=True) for td in row.find_all('td')]
                if cols:
                    data.append(cols)

            df = pd.DataFrame(data, columns=headers)

            df.to_csv(file_path, index=False) 
        except Exception as e:
            print(f'failed to scrape {full_url}')
            continue

No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=250039&set=true
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=250036&set=true
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=250029&set=true
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=250028&set=true
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti

KeyboardInterrupt: 